In [14]:
import pandas
import os
import glob
import re

In [295]:
from dateutil import parser
import datetime

In [3]:
from cStringIO import StringIO
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams

def to_txt(pdf_path):
    input_ = file(pdf_path, 'rb')
    output = StringIO()

    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    process_pdf(manager, converter, input_)

    return output.getvalue() 

In [11]:
files = glob.glob('/Users/lauren/Documents/taxes2016/check*')

In [112]:
def get_df(relevant_lines):
    df = pandas.DataFrame()
    row = {}
    for line in relevant_lines:
        if re.search('^\d{1,2}/\d{1,2}\s{2,}[A-Za-z].+\.\d\d$',line.strip()):
            if row:
                df = df.append(row, ignore_index=True)
            row = {}
            row['Date'] = parser.parse(re.search('^\d{1,2}/\d{1,2}',line.strip()).group(0)+"/2016")
            row['amount'] = float(line.strip().split(" ")[-1])
            row['description'] = line.strip()[5:].split(" on ")[0].strip()
        else:
            row['description'] = line.strip()
    df = df.append(row, ignore_index=True)
    return df

In [113]:
total = 0.0
dfs = []
for filename in files:
    txt = to_txt(filename)
    lines = txt.split("\n")
    relevant_lines = [line for line in lines if line.startswith('         ') \
                 and not line.strip().startswith('DATE ') \
                 and (re.search('^\d{1,2}/\d{1,2}\s{2,}[A-Za-z].+\.\d\d$',line.strip()) \
                      or re.search('^                [A-Z]',line))]
    amounts = [float(line.strip().split(" ")[-1]) for line in relevant_lines \
           if re.search('^\d{1,2}/\d{1,2}\s{2,}[A-Za-z].+\.\d\d$',line.strip())]
    total = total+sum(amounts)
    df = get_df(relevant_lines)
    dfs.append(df)
all_dfs = pandas.concat(dfs, ignore_index=True)

In [117]:
(all_dfs.amount.sum()-total)<1.0

True

In [293]:
all_dfs['month'] = all_dfs.Date.dt.month
relevant_dates = all_dfs[all_dfs.month.between(1,5)]
relevant_dates = relevant_dates[relevant_dates.Date<pandas.Timestamp('2016-05-01')]

In [296]:
for month in relevant_dates.month.unique():
    row = {"amount":2200, "description":"Rent","Date":datetime.datetime(2016, month, 1)}
    relevant_dates = relevant_dates.append(row, ignore_index=True)

In [302]:
relevant_dates.amount.sum()

26206.760000000002

In [301]:
def resolve_cat(s):
    s = s.strip().upper()
    s = re.sub('\s+',' ',s)
    words = s.split(' ')
    if 'FEE' in words:
        return 'Fees'
    if 'TAXI' in s or 'CAB' in words or 'CURB' in words:
        return 'Taxi'
    if 'INSURANCE' in words:
        return 'Insurance'
    if 'AMAZON' in s or 'THEATER' in s or 'BAR' in words or 'ITUNES' in s or 'AUDIBLE' in s \
    or 'LIQUOR' in words or 'WINE' in words or 'WINES' in s or 'UCB' in words or 'SING SING' in s \
    or 'TAWK' in s or 'SLATE' in s or 'STARS-' in s:
        return 'Entertainment'
    if 'MEDICAL' in s or 'PHARMA' in s:
        return 'Medical'
    if 'UAS' in words or 'ECSI' in words or 'STUDENT LN' in s or 'EDUCATIONAL COMP' in s:
        return 'Student Loans'
    if 'BROWSERSTACK' in s or 'DOCHUB' in s or 'DUANE' in words or 'RITE AID' in s or '1AND1' in s\
    or 'H BRICKMAN' in s or 'PRINTWITH' in s:
        return 'Supplies'
    if 'AT&T' in s or 'TWC' in s:
        return 'Utilities'
    if 'AIRBNB' in s:
        return 'Travel - lodging'
    if 'JFK' in s or 'INFLIGHT' in words or 'SPIRIT AI' in s:
        return 'Travel'
    if 'GIFTS' in words or 'FUNDRAZR' in s:
        return 'Charity'
    if s.startswith('MTA') or s.startswith('METRO'):
        return 'Transportation'
    if 'SMOOTHLINK' in s:
        return 'Repairs'
    if 'BOUTIQUE' in words or 'SPA' in words or 'DSW' in words or 'FLORIST' in words \
    or 'DBT PUR' in s or 'BROWARD' in s or 'CLEANERLAU' in s or 'NAILS' in words or 'CLOTHING' in words \
    or 'THRIFT' in s or 'BEAUTY' in s or 'ZARA' in words or 'SEPHORA' in words or s.startswith('STRAWBERRY') \
    or 'MATCH.COM' in s or 'RICKY`S' in s or 'FILLMORE' in words or 'LAINA' in words or 'URBANSTEMS' in s:
        return None
    if 'COFFEE' in s or 'CAFE' in s or 'DELI' in s or 'KITCHEN' in s or 'TAVERN' in words or 'MARKET' in words \
    or 'ORGANIC' in words or 'TEA' in words or 'CURRY' in words or 'CARAVAN OF' in s or 'FARFASHA' in words \
    or 'BRINKLEYS' in words or 'BAJA' in words or 'BLOSSOM ON ' in s or 'FIKA' in words or 'FRESH &' in s\
    or 'SEAMLESS' in s or 'GRUBHUB' in words or 'GRISTEDES' in words or 'DINER' in words or 'BOONSRI' in words \
    or 'DRAUGHT' in words or 'CROXLEYS' in words or 'EGG' in words or 'BISTRO' in words or 'FARM' in words \
    or 'HEART OF INDIA' in s or 'HALE AND' in s or 'DEN ' in s or s.startswith('LE ') or s.startswith('LA ') \
    or 'RESTAUR' in s or 'STARBUCKS' in s or 'THE BEAN' in s or s.startswith('THE ') or s.startswith('SQ ') \
    or s.startswith('SAN ') or 'WHOLEFDS' in s or 'LIQUITERIA' in s or 'DOJO' in words \
    or 'SPECIALTY' in words or 'SEAMLSS' in s or s.startswith('OFICINA ') or 'TACO' in words or 'TACU' in words\
    or 'M2M' in words or 'MEXICAN' in words or 'PRET' in words or 'KONDITORI' in words or 'HAIR OF THE' in s \
    or 'KOPINYC' in words or 'SWEETGREEN' in s or 'LUCIEN' in words or 'MATCHBOX CHINA' in s \
    or 'POS PURCHASE' in s or 'PIZZA' in words or 'BAMBOO' in words or 'ROASTING' in words or 'ROSIE' in s \
    or "SPICE" in words or 'SPIEGEL' in words or 'SPRING STREET NATURAL' in s or 'TRES CARNES' in s \
    or "VIC'S" in s or 'WISE MEN' in s or 'SUNAC NATURAL' in s or 'ATM WITHD' in s:
        return 'Meals'
    return s
relevant_dates['category'] = relevant_dates.description.apply(resolve_cat)
relevant_dates.groupby("category").amount.sum().sum()
relevant_dates['Month'] = relevant_dates.Date.dt.strftime('%B')
relevant_dates.drop('month', axis=1, inplace=True)

23951.499999999989

In [312]:
relevant_dates[relevant_dates.category.isnull()==False]\
.to_csv('/Users/lauren/Documents/taxes2016/items.csv', index=None)

In [316]:
relevant_dates.groupby(["category","Month"]).amount.sum()\
.to_csv('/Users/lauren/Documents/taxes2016/summary.csv')